In [1]:
rm(list=ls()) #Limpiar el entorno de trabajo
#Librerías
#if (require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse,
               foreign,
               srvyr)

In [2]:
dir.create('datos/envipe', showWarnings = FALSE)
url_1 <- 'https://www.inegi.org.mx/contenidos/programas/envipe/'
url_2 <- '/microdatos/bd_envipe_'
years <- c(2020:2024)
data_list <- list()

In [3]:
# Ciclo 1: Descarga y descompresión
for (i in 1:length(years)) {
    if (years[i] %in% c(2024)) {
        fin <- '_csv.zip'
    } else {
        fin <- '_dbf.zip'
    }

    url <- paste0(url_1, years[i], url_2, years[i], fin)
    destfile <- paste0('datos/envipe/', years[i], fin)
    folder <- paste0('datos/envipe/', years[i])

    dir.create(folder, showWarnings = FALSE)

    tryCatch(
        {
            options(timeout = 9999999)
            download.file(url, destfile, mode = 'wb')
        },
        error = function(e) {
            message('Error en la descarga: ', e)
        }
    )

    unzip_result <- tryCatch(
        {
            unzip(destfile, exdir = folder)
            TRUE
        },
        error = function(e) {
            message('Error en la descompresión: ', e)
            FALSE
        }
    )

    if (unzip_result) {
        print(paste('Archivos descomprimidos para el año', years[i]))

        # Eliminar el archivo ZIP después de la descompresión
        tryCatch(
            {
                file.remove(destfile)
                print(paste('Archivo eliminado:', destfile))
            },
            error = function(e) {
                message('Error al eliminar el archivo:', e)
            }
        )
    }
}

[1] "Archivos descomprimidos para el año 2020"
[1] "Archivo eliminado: datos/envipe/2020_dbf.zip"
[1] "Archivos descomprimidos para el año 2021"
[1] "Archivo eliminado: datos/envipe/2021_dbf.zip"
[1] "Archivos descomprimidos para el año 2022"
[1] "Archivo eliminado: datos/envipe/2022_dbf.zip"
[1] "Archivos descomprimidos para el año 2023"
[1] "Archivo eliminado: datos/envipe/2023_dbf.zip"
[1] "Archivos descomprimidos para el año 2024"
[1] "Archivo eliminado: datos/envipe/2024_csv.zip"


In [4]:
estima_inseguridad <- function(year) {

# Definir rutas
if  (year %in% c(2020:2021)) {
    ruta_base <- paste0('datos/envipe/',year,'/bd_envipe_', year, '_dbf/')
}else {
    ruta_base <- paste0('datos/envipe/', year, '/')
}


#Leer datos
if (year %in% c(2020:2023)) {
    seguridad <- read.dbf(paste0(ruta_base, 'TPer_Vic1.dbf'))

} else {
    seguridad <- read_csv(paste0(ruta_base, 'TPer_Vic1.csv'))
}

seguridad<-seguridad%>%
    janitor::clean_names()%>%
    mutate(fac_ele=as.numeric(fac_ele), sexo=as.numeric(sexo))

#Crear diseño muestral
dm<-seguridad%>%
    as_survey_design(
    weights = fac_ele,
    ids = upm_dis,
    strata = est_dis,
    nest = TRUE
    )


  #Estimar percepción de inseguridad de mujeres en la Ciudad de México
    resultado<-dm %>%
    filter(cve_ent == "09") %>%
    filter(ap4_4_09!=3)%>%
  mutate(tipo_seg=case_when(
    ap4_4_09 %in% c(1) ~ "Seguro",
    ap4_4_09 %in% c(2) ~ "Inseguro",
    ap4_4_09 %in% c(9) ~ "No sabe"))%>%
  group_by(tipo_seg) %>%
  summarise(
    pob = survey_total(vartype = "cv"),
    porcentaje = survey_prop(vartype = "cv")
  ) %>% mutate(porcentaje = porcentaje * 100)%>%
  filter(tipo_seg == "Inseguro")%>%
    mutate(year = year)

    return(resultado)
}

In [5]:
tabla_inseg<-map_dfr(2020:2024,estima_inseguridad)
tabla_inseg

New names:
• `` -> `...1`
Rows: 89630 Columns: 232
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (21): ID_VIV, ID_HOG, ID_PER, UPM, VIV_SEL, HOGAR, RESUL_H, R_SEL, AREA...
dbl (211): ...1, SEXO, EDAD, AP4_1, AP4_2_01, AP4_2_02, AP4_2_03, AP4_2_04, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


tipo_seg,pob,pob_cv,porcentaje,porcentaje_cv,year
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
Inseguro,5344241,0.01297653,87.94672,0.005921388,2020
Inseguro,5798982,0.01355732,88.25421,0.006114492,2021
Inseguro,5731359,0.01416852,86.08960,0.006677751,2022
Inseguro,5572508,0.01452879,80.67437,0.008254599,2023
Inseguro,5362354,0.01567281,77.71547,0.008917681,2024
